### **Set up**

In [1]:
import numpy as np
import glob, cv2
import os

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30 ,0.001)

# Object Point(3D)
objp = np.zeros((6*9,3),np.float32)

# Transpose to (6,9,2), flatten to reshpae(-1,2) and convert to (42,2)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3D points in the real world
imgpoints = [] # points in the 2D image

# An array to store object points and image points from the image
images = glob.glob('*.jpg')
# images = glob.glob('image5.jpg')
print(images)
path = "./storage/"
for name in images:
    img = cv2.imread(name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the corners of the chessboard
    ret, corners = cv2.findChessboardCorners(gray,(9,6),None)

    # If found, add Object points and Image points
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)

        # Let's draw a corner
        img = cv2.drawChessboardCorners(img,(9,6),corners2,ret)
#         img = cv2.resize(img, (2000,900))
        cv2.imshow(name,img)
        cv2.imwrite(os.path.join(path , name+'_corner.jpg'), img)
        cv2.waitKey(100)
        
        
cv2.destroyAllWindows()

['image9.jpg', 'image12.jpg', 'image1.jpg', 'image15.jpg', 'image4.jpg', 'image8.jpg', 'image10.jpg', 'image5.jpg', 'image3.jpg', 'image7.jpg', 'image13.jpg', 'image6.jpg', 'image14.jpg', 'image11.jpg', 'image16.jpg', 'image2.jpg']


### **Calibration**

cv.calibrateCamera() which returns the camera matrix, distortion coefficients, rotation and translation vectors etc. 

In [2]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

### **Undistortion**

Take an image

In [8]:
img = cv2.imread('image13.jpg')
h,w = img.shape[:2]
newcameraMtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

#### **Using cv.undistort()**

Call the function and use ROI obtained above to crop the result

In [9]:
# undistort
dst = cv2.undistort(img,mtx,dist,None,newcameraMtx)

#crop the image
x,y,w,h = roi
dst = dst[y:y+h,x:x+w]
cv2.imwrite('calibRes.jpg',dst) #save the result

True

#### **Using remapping**

In [10]:
img = cv2.imread('image13.jpg')
h,w = img.shape[:2]
newcameraMtx, roi = cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [11]:
# undistort
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameraMtx, (w,h), 5)
dst_remap = cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

#crop the image
x_remap, y_remap, w_remap, h_remap = roi
dst_remap = dst_remap[y_remap:y_remap+h_remap, x_remap:x_remap+w_remap]
cv2.imwrite('calibRemap.jpg',dst_remap)

True

Save the camera matrix and distortion coefficients for future use.

In [12]:
np.savez('calib.npz',ret=ret,mtx=mtx,dist=dist,rvecs=rvecs,tvecs=tvecs)

### **Re-projection Error**

In [14]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2,_ = cv2.projectPoints(objpoints[i],rvecs[i],tvecs[i],mtx,dist)
    error = cv2.norm(imgpoints[i],imgpoints2,cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print("Total error : {0}".format(mean_error/len(objpoints)))


Total error : 0.0763300239989937


In [26]:
calib_data = np.load('calib.npz')
cmx = calib_data['mtx']
dist = calib_data['dist']
print("matrice de caméra: \n",cmx)
print("\n")
print("coefficients de distorsion: \n",dist)

matrice de caméra: 
 [[2.94330979e+03 0.00000000e+00 2.03022456e+03]
 [0.00000000e+00 2.94972887e+03 8.89761014e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


coefficients de distorsion: 
 [[ 0.10179749 -0.02986528 -0.00337707  0.00060471 -0.58474945]]


In [16]:
from PIL import Image

im = Image.open('image2.jpg')
image_size = im.size
print(image_size)

(4000, 1800)


In [19]:
import cv2
fov_x, fov_y, focal_len, principal, aspect = cv2.calibrationMatrixValues(cmx, image_size, 6.4,4.8)
print(fov_x, fov_y, focal_len, principal, aspect)

68.38893230041549 33.93488343921044 4.709295667172504 (3.248359300026061, 2.3726960383740163) 1.0021809042520036
